In [ ]:
import pandas as pd
import polars as pl
# This file is part of the Open Data Cube project
from data_processing.data_processing_strategies import read_hdf
import glob

In [ ]:
from typing import List, Dict, Any, TypedDict
from enum import Enum

class ProjectType(str, Enum):
    D2 = "2d"
    D3 = "3d"


class DataProcessingStrategy(TypedDict):
    name: str
    parameters: Dict[str, Any]



class DataProcessingOutput(TypedDict):
    strategy: DataProcessingStrategy
    result: pl.DataFrame | None
    completed: bool


class VideoData2D(TypedDict):
    video_path: str | None
    dlc_path: str
    original_dlc_data: pl.DataFrame
    processed_dlc_data: list[DataProcessingOutput]


class VideoData3D(TypedDict):
    video_paths: List[str]
    dlc_data: pl.DataFrame


class ProjectData(TypedDict):
    project_path: str
    project_name: str
    video_data: List[VideoData2D] | List[VideoData3D]
    project_type: ProjectType
    data_processing_strategies: List[DataProcessingStrategy]


In [4]:
from pathlib import Path
def convert_str_to_paths(video_paths: List[str]) -> List[Path]:
    """
    Convert a list of string paths to Path objects.
    """
    return [Path(path) for path in video_paths]

In [5]:
import os
from pathlib import Path

# Paths
project_path = "/home/thomas/Documents/DeepLabCutClusteringV3/data/preprocessed/hugo_project"
dlc_h5_paths = convert_str_to_paths(glob.glob(f"{project_path}/dlc_data/*.h5"))
video_paths = convert_str_to_paths(glob.glob(f"{project_path}/videos/*"))

project_data = ProjectData(
    project_path=project_path,
    project_name="hugo_project",
    video_data=[],
    project_type=ProjectType.D2,
    data_processing_strategies=[]
)


video_dir = video_paths[0].parent
h5_to_video_map = {}
for dlc_h5_path in dlc_h5_paths:
    video_map_path = video_dir / (dlc_h5_path.stem + ".avi")
    h5_to_video_map[dlc_h5_path] = video_map_path



In [6]:
for delc_path in h5_to_video_map.keys():
    video_path = h5_to_video_map[delc_path]
    if not video_path.exists():
        print(f"Video file {video_path} does not exist for DLC data {delc_path}.")
        video_path = None

    original_dlc_data = read_hdf(str(delc_path))
    
    video_data_2d = VideoData2D(
        video_path=str(video_path),
        dlc_path=str(dlc_h5_path),
        original_dlc_data=original_dlc_data,
        processed_dlc_data=[]
    )
    
    project_data['video_data'].append(video_data_2d)

In [ ]:
def KeepOriginalStrategy(df, include_likelyhood=False):
    """Clones the dataframe, filters the columns to only include those that end with _x, _y, _z, and optionally _likelihood."""
    if include_likelyhood:
        regex = "^.*(_x|_y|_z|_likelihood)$"
    elif include_likelyhood == False:
        regex = "^.*(_x|_y|_z)$"


    filtered = df.select(pl.col(regex))

    return filtered


In [117]:

def CentroidDiffStrategy(df: pl.DataFrame, include_centroid: bool = False) -> pl.DataFrame:
    """Finds the centroid difference for each _x, _y, (_z) column."""

    df = df.select([col for col in df.columns if not col.endswith("_likelihood")])
    # Add centroid columns
    df = df.with_columns(
        centroid_x=pl.concat_list(pl.col("^.*_x$")).list.mean(),
        centroid_y=pl.concat_list(pl.col("^.*_y$")).list.mean(),
    )

    if df.select(pl.col("^.*_z$")).width > 0:
        df = df.with_columns(
            centroid_z=pl.concat_list(pl.col("^.*_z$")).list.mean(),
        )

    # Subtract centroid from matching columns
    for axis in ['x', 'y', 'z']:

        axis_cols = df.select(pl.col(f"^.*_{axis}$")).columns

        if not axis_cols:
            continue

        df = df.with_columns([
            (pl.col(col) - pl.col(f"centroid_{axis}")).alias(col) for col in axis_cols
        ])

    # Optionally drop centroid columns
    if not include_centroid:
        df = df.drop(["centroid_x", "centroid_y", "centroid_z"], strict=False)

    return df


In [118]:
import polars as pl

def VelocityStrategy(df: pl.DataFrame) -> pl.DataFrame:
    """Returns a DataFrame containing only the velocity (Euclidean) for each body part."""

    # Identify all body part prefixes (excluding _likelihood)
    body_parts = set()
    for col in df.columns:
        if col.endswith("_x") and not col.endswith("_likelihood"):
            body_parts.add(col[:-2])

    velocity_exprs = []

    for part in body_parts:
        x_col = f"{part}_x"
        y_col = f"{part}_y"
        has_z = f"{part}_z" in df.columns

        dx = pl.col(x_col).diff()
        dy = pl.col(y_col).diff()

        if has_z:
            dz = pl.col(f"{part}_z").diff()
            velocity = (dx**2 + dy**2 + dz**2).sqrt().alias(f"{part}_velocity")
        else:
            velocity = (dx**2 + dy**2).sqrt().alias(f"{part}_velocity")

        velocity_exprs.append(velocity)

    # Return only velocity columns
    df = df.select(velocity_exprs)
    df = df.fill_null(strategy="backward")
    df = df.fill_null(strategy="forward")

    return df


In [119]:
project_data["data_processing_strategies"] = [VelocityStrategy, CentroidDiffStrategy]


In [125]:
for video in project_data["video_data"]:
    dfs = []
    for strategy in project_data["data_processing_strategies"]:
        dfs.append(strategy(video["original_dlc_data"]))

    df_combined = pl.concat(dfs, how="horizontal")

In [126]:
df_combined

nose_velocity,neck_end_velocity,throat_end_velocity,back_left_knee_velocity,front_left_paw_velocity,front_right_paw_velocity,lower_jaw_velocity,body_middle_left_velocity,left_earbase_velocity,upper_jaw_velocity,back_middle_velocity,neck_base_velocity,mouth_end_left_velocity,right_antler_base_velocity,right_eye_velocity,front_left_knee_velocity,left_antler_base_velocity,tail_end_velocity,right_earend_velocity,back_end_velocity,belly_bottom_velocity,front_right_knee_velocity,back_right_knee_velocity,tail_base_velocity,right_antler_end_velocity,back_left_paw_velocity,right_earbase_velocity,back_base_velocity,mouth_end_right_velocity,back_right_thai_velocity,back_right_paw_velocity,body_middle_right_velocity,left_eye_velocity,front_left_thai_velocity,back_left_thai_velocity,front_right_thai_velocity,left_antler_end_velocity,…,back_end_y,back_middle_x,back_middle_y,tail_base_x,tail_base_y,tail_end_x,tail_end_y,front_left_thai_x,front_left_thai_y,front_left_knee_x,front_left_knee_y,front_left_paw_x,front_left_paw_y,front_right_thai_x,front_right_thai_y,front_right_knee_x,front_right_knee_y,front_right_paw_x,front_right_paw_y,back_left_paw_x,back_left_paw_y,back_left_thai_x,back_left_thai_y,back_right_thai_x,back_right_thai_y,back_left_knee_x,back_left_knee_y,back_right_knee_x,back_right_knee_y,back_right_paw_x,back_right_paw_y,belly_bottom_x,belly_bottom_y,body_middle_right_x,body_middle_right_y,body_middle_left_x,body_middle_left_y
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2.664108,28.227034,22.373381,2.139955,9.474257,3.006879,11.801864,14.553572,28.498438,12.273447,11.504195,11.618459,12.179778,11.711031,12.039073,23.364204,11.711031,19.950373,11.523703,11.030224,28.132244,2.734654,11.111619,11.030224,19.950373,2.244853,11.711031,28.227034,6.85158,11.614697,11.319461,14.553572,11.945323,9.411612,2.109274,12.183305,19.950373,…,196.153839,31.384613,62.278839,136.572113,196.153839,251.322113,186.591339,-25.990387,-62.033661,-83.365387,-23.783661,-121.615387,-42.908661,-112.052887,-90.721161,-121.615387,-71.596161,-178.990387,-90.721161,174.822113,387.403839,107.884613,243.966339,21.822113,263.091339,136.572113,301.341339,60.072113,282.216339,88.759613,330.028839,-83.365387,119.653839,-16.427887,90.966339,-16.427887,90.966339
2.664108,28.227034,22.373381,2.139955,9.474257,3.006879,11.801864,14.553572,28.498438,12.273447,11.504195,11.618459,12.179778,11.711031,12.039073,23.364204,11.711031,19.950373,11.523703,11.030224,28.132244,2.734654,11.111619,11.030224,19.950373,2.244853,11.711031,28.227034,6.85158,11.614697,11.319461,14.553572,11.945323,9.411612,2.109274,12.183305,19.950373,…,191.846954,32.93869,58.409454,137.610565,191.846954,261.31369,182.315704,-33.670685,-55.965546,-71.733185,-17.840546,-128.826935,-36.903046,-109.795685,-94.090546,-128.826935,-75.028046,-185.920685,-94.090546,166.15744,382.471954,99.548065,239.503204,23.423065,258.565704,128.09494,296.690704,42.454315,287.159454,90.03244,325.284454,-71.733185,134.659454,-14.639435,96.534454,-14.639435,96.534454
22.453146,4.294943,13.075988,5.846945,6.417292,9.443306,22.390821,32.770657,13.626756,31.968765,23.507214,23.849894,12.93784,23.57346,51.437508,15.585301,30.120676,44.174957,5.346034,14.770279,22.861704,12.832644,7.826503,14.770279,44.174957,6.370039,13.562824,13.831188,31.968765,17.135775,5.937335,32.770657,22.828146,11.128171,5.49087,12.951118,44.174957,…,192.120987,40.768433,57.808487,136.549683,192.120987,289.799683,182.527237,-45.434692,-47.722763,-102.903442,-18.941513,-150.794067,-38.129013,-112.481567,-95.691513,-131.637817,-76.504013,-198.684692,-105.285263,155.705933,383.995972,88.659058,240.089737,2.455933,240.089737,117.393433,297.652222,31.190308,278.464722,69.502808,326.433472,-64.590942,134.558487,2.4

In [115]:
project_data["video_data"][0]

{'video_path': '/home/thomas/Documents/DeepLabCutClusteringV3/data/preprocessed/hugo_project/videos/1-1_M1R_3d_camA.avi',
 'dlc_path': '/home/thomas/Documents/DeepLabCutClusteringV3/data/preprocessed/hugo_project/dlc_data/1-1_M1R_3d_camA.h5',
 'original_dlc_data': shape: (18_019, 117)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ nose_x    ┆ nose_y    ┆ nose_like ┆ upper_jaw ┆ … ┆ body_midd ┆ body_midd ┆ body_midd ┆ body_mid │
 │ ---       ┆ ---       ┆ lihood    ┆ _x        ┆   ┆ le_right_ ┆ le_left_x ┆ le_left_y ┆ dle_left │
 │ f32       ┆ f32       ┆ ---       ┆ ---       ┆   ┆ likelihoo ┆ ---       ┆ ---       ┆ _likelih │
 │           ┆           ┆ f32       ┆ f32       ┆   ┆ d         ┆ f32       ┆ f32       ┆ ood      │
 │           ┆           ┆           ┆           ┆   ┆ ---       ┆           ┆           ┆ ---      │
 │           ┆           ┆           ┆           ┆   ┆ f32       ┆           ┆           ┆ f32      │


In [105]:
for video_data in project_data["video_data"]:
    df = video_data["original_dlc_data"]
    

In [106]:
df = VelocityStrategy(df)

In [107]:
df

nose_velocity,neck_end_velocity,throat_end_velocity,back_left_knee_velocity,front_left_paw_velocity,front_right_paw_velocity,lower_jaw_velocity,body_middle_left_velocity,left_earbase_velocity,upper_jaw_velocity,back_middle_velocity,neck_base_velocity,mouth_end_left_velocity,right_antler_base_velocity,right_eye_velocity,front_left_knee_velocity,left_antler_base_velocity,tail_end_velocity,right_earend_velocity,back_end_velocity,belly_bottom_velocity,front_right_knee_velocity,back_right_knee_velocity,tail_base_velocity,right_antler_end_velocity,back_left_paw_velocity,right_earbase_velocity,back_base_velocity,mouth_end_right_velocity,back_right_thai_velocity,back_right_paw_velocity,body_middle_right_velocity,left_eye_velocity,front_left_thai_velocity,back_left_thai_velocity,front_right_thai_velocity,left_antler_end_velocity,throat_base_velocity,left_earend_velocity
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2.664108,28.227034,22.373381,2.139955,9.474257,3.006879,11.801864,14.553572,28.498438,12.273447,11.504195,11.618459,12.179778,11.711031,12.039073,23.364204,11.711031,19.950373,11.523703,11.030224,28.132244,2.734654,11.111619,11.030224,19.950373,2.244853,11.711031,28.227034,6.85158,11.614697,11.319461,14.553572,11.945323,9.411612,2.109274,12.183305,19.950373,6.994299,20.992332
2.664108,28.227034,22.373381,2.139955,9.474257,3.006879,11.801864,14.553572,28.498438,12.273447,11.504195,11.618459,12.179778,11.711031,12.039073,23.364204,11.711031,19.950373,11.523703,11.030224,28.132244,2.734654,11.111619,11.030224,19.950373,2.244853,11.711031,28.227034,6.85158,11.614697,11.319461,14.553572,11.945323,9.411612,2.109274,12.183305,19.950373,6.994299,20.992332
22.453146,4.294943,13.075988,5.846945,6.417292,9.443306,22.390821,32.770657,13.626756,31.968765,23.507214,23.849894,12.93784,23.57346,51.437508,15.585301,30.120676,44.174957,5.346034,14.770279,22.861704,12.832644,7.826503,14.770279,44.174957,6.370039,13.562824,13.831188,31.968765,17.135775,5.937335,32.770657,22.828146,11.128171,5.49087,12.951118,44.174957,13.004544,10.680199
33.179173,38.979507,38.900639,19.712759,8.119927,99.160576,47.651257,25.979815,32.12096,33.179173,23.712339,29.745972,52.219578,41.110371,24.881371,14.454962,32.12096,46.190945,49.416424,45.986618,30.045647,104.065872,15.749166,45.986618,46.190945,8.231521,56.992519,36.034794,41.918255,53.493275,4.525387,25.979815,42.331455,25.072018,3.379211,122.089714,46.190945,49.846058,20.005983
49.79694,46.407078,54.370296,19.397928,107.684334,91.833565,64.107925,62.588787,40.779572,73.560287,39.430069,17.130297,47.772171,47.972736,56.986393,78.555916,40.779572,23.567719,44.56831,50.751717,54.424797,66.919067,25.226509,50.751717,23.567719,0.406025,47.972736,46.407078,75.871239,22.688917,8.145486,62.588787,55.943665,47.222454,72.841148,60.975365,23.567719,49.15461,40.426754
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
24.159718,12.547578,17.187473,0.79584,11.910875,18.542019,24.122826,12.116452,18.984442,30.570768,8.786743,18.298178,16.07815,19.096416,17.161715,7.606764,18.984442,4.8046875,18.932035,10.20851,10.867872,18.542019,11.243111,10.20851,4.8046875,1.785374,19.096416,12.547578,24.327188,7.170875,1.519104,12.116452,16.947575,8.317525,8.062504,28.580843,4.8046875,16.314074,19.730577
18.803114,11.731254,13.127836,20.029013,19.404375,10.484386,18.799219,9.582725,17.390688,20.668213,11.215192,19.090586,28.103226,18.808664,27.734533,12.906335,17.390688,6.897159,18.898403,6.919547,10.503929,10.484386,20.272827,6.919547,6.897159,7.804074,18.808664,11.731254,7.963433,128.873886,83.327675,9.582725,27.528542,1.581911,15.422453,17.95558,6.897159,10.109484,8.60973
13.99374,7.104931,8.53576,10.530789,15.02373,13.250316,5.362432,7.606234,15.571747,5.455055,6.795586,6.925564,21.9097,15.53907,12.934756,4.352404,15.571747,9.461983,6.438149,2.770663,3.996558,13.250316,4.576985,2.770663,9.461983

In [39]:
df = df.with_columns(
    centroid_x=pl.concat_list(pl.col("^.*_x$")).list.mean(),
)
df = df.with_columns(
    centroid_y=pl.concat_list(pl.col("^.*_y$")).list.mean(),
)
# df = df.with_columns(
#     centroid_z=pl.concat_list(pl.col("^.*_z$")).list.mean(),
# )

In [30]:
centroid_x = pl.concat_list(pl.col("^.*_x$")).list.mean()

In [31]:
centroid_x

<Expr ['col("^.*_x$").list.concat().li…'] at 0x7D5681C0E930>

In [ ]:
for dimension in ["_x", "_y", "_z"]:
    


nose_x,nose_y,nose_likelihood,upper_jaw_x,upper_jaw_y,upper_jaw_likelihood,lower_jaw_x,lower_jaw_y,lower_jaw_likelihood,mouth_end_right_x,mouth_end_right_y,mouth_end_right_likelihood,mouth_end_left_x,mouth_end_left_y,mouth_end_left_likelihood,right_eye_x,right_eye_y,right_eye_likelihood,right_earbase_x,right_earbase_y,right_earbase_likelihood,right_earend_x,right_earend_y,right_earend_likelihood,right_antler_base_x,right_antler_base_y,right_antler_base_likelihood,right_antler_end_x,right_antler_end_y,right_antler_end_likelihood,left_eye_x,left_eye_y,left_eye_likelihood,left_earbase_x,left_earbase_y,left_earbase_likelihood,left_earend_x,…,front_left_paw_likelihood,front_right_thai_x,front_right_thai_y,front_right_thai_likelihood,front_right_knee_x,front_right_knee_y,front_right_knee_likelihood,front_right_paw_x,front_right_paw_y,front_right_paw_likelihood,back_left_paw_x,back_left_paw_y,back_left_paw_likelihood,back_left_thai_x,back_left_thai_y,back_left_thai_likelihood,back_right_thai_x,back_right_thai_y,back_right_thai_likelihood,back_left_knee_x,back_left_knee_y,back_left_knee_likelihood,back_right_knee_x,back_right_knee_y,back_right_knee_likelihood,back_right_paw_x,back_right_paw_y,back_right_paw_likelihood,belly_bottom_x,belly_bottom_y,belly_bottom_likelihood,body_middle_right_x,body_middle_right_y,body_middle_right_likelihood,body_middle_left_x,body_middle_left_y,body_middle_left_likelihood
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
190.34375,4.78125,0.357366,171.21875,4.78125,0.399968,190.34375,23.90625,0.535817,190.34375,4.78125,0.461428,190.34375,14.34375,0.505818,219.03125,4.78125,0.179249,285.96875,14.34375,0.584802,324.21875,23.90625,0.215253,285.96875,14.34375,0.668526,553.71875,368.15625,0.553569,238.15625,4.78125,0.332158,285.96875,14.34375,0.725044,333.78125,…,0.894812,190.34375,90.84375,0.810543,180.78125,109.96875,0.809678,123.40625,90.84375,1.0,477.21875,568.96875,1.0,410.28125,425.53125,0.900944,324.21875,444.65625,0.817142,438.96875,482.90625,0.961541,362.46875,463.78125,0.690408,391.15625,511.59375,1.0,219.03125,301.21875,0.349726,285.96875,272.53125,0.366315,285.96875,272.53125,0.364136
193.007812,4.765625,0.299516,183.492188,4.765625,0.42101,183.492188,14.296875,0.634462,183.492188,4.765625,0.447349,202.523438,14.296875,0.673695,231.070312,4.765625,0.151945,297.679688,14.296875,0.64389,335.742188,23.828125,0.313763,297.679688,14.296875,0.72387,573.632812,366.953125,0.520749,250.101562,4.765625,0.420117,307.195312,33.359375,0.72924,354.773438,…,1.0,202.523438,90.546875,0.84075,183.492188,109.609375,0.844054,126.398438,90.546875,1.0,478.476562,567.109375,1.0,411.867188,424.140625,0.837368,335.742188,443.203125,0.833521,440.414062,481.328125,0.995627,354.773438,471.796875,0.802889,402.351562,509.921875,1.0,240.585938,319.296875,0.602081,297.679688,281.171875,0.440591,297.679688,281.171875,0.437999
215.460938,4.796875,0.344737,215.460938,4.796875,0.52707,205.882812,14.390625,0.74023,215.460938,4.796875,0.557477,215.460938,14.390625,0.675476,282.507812,4.796875,0.256158,311.242188,14.390625,0.639637,330.398438,23.984375,0.477865,311.242188,33.578125,0.705713,617.742188,369.359375,0.51788,272.929688,4.796875,0.645129,320.820312,33.578125,0.764062,359.132812,…,1.0,215.460938,91.140625,0.702005,196.304688,110.328125,0.825106,129.257812,81.546875,1.0,483.648438,570.828125,1.0,416.601562,426.921875,0.834502,330.398438,426.921875,0.677227,445.335938,484.484375,0.987816,359.132812,465.296875,0.786037,397.445312,513.265625,1.0,263.351562,321.390625,0.816367,330.398438,283.015625,0.470637,330.398438,283.015625,0.467526
220.828125,37.539062,1.0,220.828125,37.539062,1.0,229.859375,55.570312,0.899582,211.796875,46.554688,0.341739,229.859375,64.585938,1.0,275.015625,28.52

In [14]:
video_data["original_dlc_data"]

nose_x,nose_y,nose_likelihood,upper_jaw_x,upper_jaw_y,upper_jaw_likelihood,lower_jaw_x,lower_jaw_y,lower_jaw_likelihood,mouth_end_right_x,mouth_end_right_y,mouth_end_right_likelihood,mouth_end_left_x,mouth_end_left_y,mouth_end_left_likelihood,right_eye_x,right_eye_y,right_eye_likelihood,right_earbase_x,right_earbase_y,right_earbase_likelihood,right_earend_x,right_earend_y,right_earend_likelihood,right_antler_base_x,right_antler_base_y,right_antler_base_likelihood,right_antler_end_x,right_antler_end_y,right_antler_end_likelihood,left_eye_x,left_eye_y,left_eye_likelihood,left_earbase_x,left_earbase_y,left_earbase_likelihood,left_earend_x,…,front_left_paw_likelihood,front_right_thai_x,front_right_thai_y,front_right_thai_likelihood,front_right_knee_x,front_right_knee_y,front_right_knee_likelihood,front_right_paw_x,front_right_paw_y,front_right_paw_likelihood,back_left_paw_x,back_left_paw_y,back_left_paw_likelihood,back_left_thai_x,back_left_thai_y,back_left_thai_likelihood,back_right_thai_x,back_right_thai_y,back_right_thai_likelihood,back_left_knee_x,back_left_knee_y,back_left_knee_likelihood,back_right_knee_x,back_right_knee_y,back_right_knee_likelihood,back_right_paw_x,back_right_paw_y,back_right_paw_likelihood,belly_bottom_x,belly_bottom_y,belly_bottom_likelihood,body_middle_right_x,body_middle_right_y,body_middle_right_likelihood,body_middle_left_x,body_middle_left_y,body_middle_left_likelihood
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
190.34375,4.78125,0.357366,171.21875,4.78125,0.399968,190.34375,23.90625,0.535817,190.34375,4.78125,0.461428,190.34375,14.34375,0.505818,219.03125,4.78125,0.179249,285.96875,14.34375,0.584802,324.21875,23.90625,0.215253,285.96875,14.34375,0.668526,553.71875,368.15625,0.553569,238.15625,4.78125,0.332158,285.96875,14.34375,0.725044,333.78125,…,0.894812,190.34375,90.84375,0.810543,180.78125,109.96875,0.809678,123.40625,90.84375,1.0,477.21875,568.96875,1.0,410.28125,425.53125,0.900944,324.21875,444.65625,0.817142,438.96875,482.90625,0.961541,362.46875,463.78125,0.690408,391.15625,511.59375,1.0,219.03125,301.21875,0.349726,285.96875,272.53125,0.366315,285.96875,272.53125,0.364136
193.007812,4.765625,0.299516,183.492188,4.765625,0.42101,183.492188,14.296875,0.634462,183.492188,4.765625,0.447349,202.523438,14.296875,0.673695,231.070312,4.765625,0.151945,297.679688,14.296875,0.64389,335.742188,23.828125,0.313763,297.679688,14.296875,0.72387,573.632812,366.953125,0.520749,250.101562,4.765625,0.420117,307.195312,33.359375,0.72924,354.773438,…,1.0,202.523438,90.546875,0.84075,183.492188,109.609375,0.844054,126.398438,90.546875,1.0,478.476562,567.109375,1.0,411.867188,424.140625,0.837368,335.742188,443.203125,0.833521,440.414062,481.328125,0.995627,354.773438,471.796875,0.802889,402.351562,509.921875,1.0,240.585938,319.296875,0.602081,297.679688,281.171875,0.440591,297.679688,281.171875,0.437999
215.460938,4.796875,0.344737,215.460938,4.796875,0.52707,205.882812,14.390625,0.74023,215.460938,4.796875,0.557477,215.460938,14.390625,0.675476,282.507812,4.796875,0.256158,311.242188,14.390625,0.639637,330.398438,23.984375,0.477865,311.242188,33.578125,0.705713,617.742188,369.359375,0.51788,272.929688,4.796875,0.645129,320.820312,33.578125,0.764062,359.132812,…,1.0,215.460938,91.140625,0.702005,196.304688,110.328125,0.825106,129.257812,81.546875,1.0,483.648438,570.828125,1.0,416.601562,426.921875,0.834502,330.398438,426.921875,0.677227,445.335938,484.484375,0.987816,359.132812,465.296875,0.786037,397.445312,513.265625,1.0,263.351562,321.390625,0.816367,330.398438,283.015625,0.470637,330.398438,283.015625,0.467526
220.828125,37.539062,1.0,220.828125,37.539062,1.0,229.859375,55.570312,0.899582,211.796875,46.554688,0.341739,229.859375,64.585938,1.0,275.015625,28.52